# Import necessary packages

In [1]:
import pathlib
import glob
import os

import matplotlib.pyplot as plt

from helpers.Vehicle import Vehicle
from helpers.Gaze import Gaze

# Set variables and prepare data

In [2]:
RUN_IDX = 6 # e.g., run_{RUN_IDX}
N_LAPS = 10

SAVE_DIR = pathlib.Path(".").joinpath(f"run_{RUN_IDX}").absolute()
GAZE_DIR = pathlib.Path(max(glob.glob(os.path.join(SAVE_DIR, '*/')), key=os.path.getmtime))

PATH_GAZE = GAZE_DIR.joinpath("gaze_positions.csv")
PATH_BLINK = GAZE_DIR.joinpath("blinks.csv")

PATH_VEHICLE = SAVE_DIR.joinpath("df_vehicle.csv")

print(PATH_GAZE)
print(PATH_VEHICLE)

gaze = Gaze(PATH_GAZE, PATH_BLINK)
vehicle = Vehicle(PATH_VEHICLE, N_LAPS)

/Users/jhpark/Desktop/BAAR/run_6/001/gaze_positions.csv
/Users/jhpark/Desktop/BAAR/run_6/df_vehicle.csv


In [ ]:
veh_lap_end_idx = vehicle.find_veh_lap_end_idx()
gaze_lap_end_idx = vehicle.find_gaze_lap_end_idx(gaze.get_df())

print(veh_lap_end_idx)
print(gaze_lap_end_idx)

# Visualize data

In [ ]:
def get_intervals(idx):
    idx_lst = []
    for i in range(len(idx)):
        if i == 0:
            idx_lst.append((0, idx[i]))
        else:   
            idx_lst.append((idx[i-1]+1, idx[i]))

    return idx_lst

lap_idx = get_intervals(veh_lap_end_idx)
gaze_lap_idx = get_intervals(gaze_lap_end_idx)

df_vehicle = vehicle.get_df()
df_gaze = gaze.get_df()

### $\color{red}{\text{Might have to look into see why there is difference in length}}$

might need to cut off at the end

In [ ]:
print(vehicle.get_df()["time_real"].describe())
print(gaze.get_df()["gaze_timestamp"].describe())

### Choose section

In [ ]:
section_lst = ['stop_1', 'sharp_cor_1', 'round_cor_1', 'round_cor_2', 'sharp_cor_2', 'stop_2']
section = section_lst[3]
section = ""

In [ ]:
RES_SAVE_DIR = pathlib.Path("./results").joinpath(f"run_{RUN_IDX}").absolute()

## Distance domain normalization

### Action (1. Brake pressure, 2. Gas (Pos. of Acc Pedal), 3. Steering angle) & Forward velocity

In [ ]:
column_labels = ['Brake[Bar]_avg', 'Pos_AccPedal___', 'Angle_SteeringWheel_deg_', 'v_x_Vehicle_CoG[km|h]']

# fig.suptitle('Action in distance domain before stop', fontsize=16)

for section in section_lst:
    print(f"processing {section} ...")
    fig, axs = plt.subplots(N_LAPS, len(column_labels), figsize=(20, 20))
    
    for i in range(N_LAPS):

        # sectioned interval. (e.g., turning corner, section before/after stop sign, etc.)
        df_section, section_start_idx, section_end_idx = vehicle.get_section(i, section)

        # x = None
        # if "stop" in section:
        #     norm_idx = vehicle.find_stop_point(df_section, e=0.5) # normalization w.r.t. speed < e
        #     x = df_section['s']-df_section['s'][norm_idx]
        # else:
        #     x = df_section['s']-df_section['s'][section_start_idx]
        x = df_section['s'] - df_section['s'][section_start_idx]

        print(f"Lap {i+1}: Completed")

        # Column data
        brake_pressure = df_section[column_labels[0]]
        v_x            = df_section[column_labels[1]]
        a_y            = df_section[column_labels[2]]
        yaw            = df_section[column_labels[3]]
        vline          = 0 # df_vehicle['x'][norm_idx]
    
        # Plot brake pressure
        axs[i, 0].plot(x, brake_pressure, label=f'Lap {i+1}')
        axs[i, 0].set_title(f'Lap {i+1} - Brake[Bar]_avg')
        axs[i, 0].axvline(x=vline, color='red', linestyle='--')
    
        # Plot velocity
        axs[i, 1].plot(x, v_x, label=f'Lap {i+1}')
        axs[i, 1].set_title(f'Lap {i+1} - Acceleration Pedal Position')
        axs[i, 1].axvline(x=vline, color='red', linestyle='--')

        # Plot acceleration
        axs[i, 2].plot(x, a_y, label=f'Lap {i+1}')
        axs[i, 2].set_title(f'Lap {i+1} - Steering Wheel Angle [deg]')
        axs[i, 2].axvline(x=vline, color='red', linestyle='--')

        #Plot Yaw rate
        axs[i, 3].plot(x, yaw, label=f'Lap {i+1}')
        axs[i, 3].set_title(f'Lap {i+1} - v_x_Vehicle_CoG[km|h]')
        axs[i, 3].axvline(x=vline, color='red', linestyle='--')

        # Set common labels
        for j in range(len(column_labels)):
            axs[i, j].set_xlabel('x (m)')

    axs[5, 0].set_ylabel('Mean Brake Pressure Across 4 Wheels [Bar]', fontsize = 16)
    axs[5, 1].set_ylabel('Gas positions', fontsize = 16)
    axs[5, 2].set_ylabel('Steering wheel angle [deg]', fontsize = 16)
    axs[5, 3].set_ylabel('Center of gravity x-velocity [Km/h]', fontsize = 16)

    # Adjust layout
    plt.tight_layout()

    # Show the plot
    # plt.show()

    # fig.savefig("dist_action.png")
    plt.savefig(RES_SAVE_DIR.joinpath(f"dist_action_{section}.png"))
    print(f"saving {section} done. \n")

In [ ]:
column_labels = ['Brake[Bar]_avg', 'a_y_Vehicle_CoG[m|s2]', 'YawRate_Vehicle_CoG[deg|s]', 'v_x_Vehicle_CoG[km|h]']


fig, axs = plt.subplots(N_LAPS, len(column_labels), figsize=(20, 20))
# fig.suptitle('Vehicle motion in distance domain before stop', fontsize=16)

for i in range(N_LAPS):
    
    # lap interval
    start_idx, end_idx = lap_idx[i][0], lap_idx[i][1]
    gaze_start_idx, gaze_end_idx = gaze_lap_idx[i][0], gaze_lap_idx[i][1]
    norm_idx = find_stop_point(df_vehicle, start_idx, end_idx, e=0.5) # normalization w.r.t. speed < e
    print("Normalized index:", norm_idx)
    # ts_vehicle = df_vehicle['time_real'].iloc(norm_idx)
    # gaze_norm_idx = gaze.find_closest_ts_idx(ts_vehicle)
    
    # sectioned interval. (e.g., turning corner, section before/after stop sign, etc.)
    start_idx, end_idx = section_idx(start_idx, end_idx)
    gaze_start_idx, gaze_end_idx = section_idx(gaze_start_idx, gaze_end_idx)

    x = df_vehicle['s'][start_idx:end_idx] - df_vehicle['s'][norm_idx]

    # start_idx = x[(x > -50) & (x < 5)].index[0]
    # end_idx = x[(x > -50) & (x < 5)].index[-1]
    # Condition to filter the DataFrame
    condition = (df_vehicle['ego_x'] < 100) & (df_vehicle['ego_y'] > 60)
    indices = df_vehicle.index[condition]
    start_index = indices.min()
    end_index = indices.max()

    x = df_vehicle['s'][start_idx:end_idx] - df_vehicle['s'][norm_idx]

    print(f"Lap {i+1}: {df_vehicle['s'][norm_idx]}")
    # print(x)

    # Column data
    brake_pressure = df_vehicle['Brake[Bar]_avg'][start_idx:end_idx]
    v_x            = df_vehicle['v_x_Vehicle_CoG[km|h]'][start_idx:end_idx]
    a_y            = df_vehicle['a_y_Vehicle_CoG[m|s2]'][start_idx:end_idx]
    yaw            = df_vehicle['YawRate_Vehicle_CoG[deg|s]'][start_idx:end_idx]
    vline          = 0 # df_vehicle['x'][norm_idx]

    # Plot brake pressure
    axs[i, 0].plot(x, brake_pressure, label=f'Lap {i+1}')
    axs[i, 0].set_title(f'Lap {i+1} - stop sign interval braking behavior')
    axs[i, 0].axvline(x=vline, color='red', linestyle='--')
  
    # Plot velocity
    axs[i, 1].plot(x, v_x, label=f'Lap {i+1}')
    axs[i, 1].set_title(f'Lap {i+1} - X-direction velocity [km/h]')
    axs[i, 1].axvline(x=vline, color='red', linestyle='--')

    # Plot acceleration
    axs[i, 2].plot(x, a_y, label=f'Lap {i+1}')
    axs[i, 2].set_title(f'Lap {i+1} - Y-direction acceleration [m/s^2]')
    axs[i, 2].axvline(x=vline, color='red', linestyle='--')

    #Plot Yaw rate
    axs[i, 3].plot(x, yaw, label=f'Lap {i+1}')
    axs[i, 3].set_title(f'Lap {i+1} - Yaw rate [deg/s]')
    axs[i, 3].axvline(x=vline, color='red', linestyle='--')

    # Set common labels
    for j in range(len(column_labels)):
        axs[i, j].set_xlabel('x (m)')

    print()

axs[5, 0].set_ylabel('Mean Brake Pressure Across 4 Wheels [Bar]', fontsize = 16)
axs[5, 1].set_ylabel('center of gravity x-velocity [Km/h]', fontsize = 16)
axs[5, 2].set_ylabel('center of gravity y-acceleration [m/(s^2)]', fontsize = 16)
axs[5, 3].set_ylabel('center of gravity yaw rate [deg/s]', fontsize = 16)

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

plt.savefig("dist_veh_motion.png")

## Time domain normalization

In [ ]:
column_labels = ['Brake[Bar]_avg', 'a_y_Vehicle_CoG[m|s2]', 'YawRate_Vehicle_CoG[deg|s]', 'v_x_Vehicle_CoG[km|h]']

fig, axs = plt.subplots(N_LAPS, len(column_labels), figsize=(20, 20))

for i in range(N_LAPS):
    
    # lap interval
    start_idx, end_idx = lap_idx[i][0], lap_idx[i][1]
    gaze_start_idx, gaze_end_idx = gaze_lap_idx[i][0], gaze_lap_idx[i][1]
    norm_idx = find_stop_point(df_vehicle, start_idx, end_idx, e=0.5) # normalization w.r.t. speed < e
    
    print("Normalized index:", norm_idx)
    ts_vehicle = df_vehicle['time_real'][norm_idx]
    gaze_norm_idx = gaze.find_closest_ts_idx(ts_vehicle)
    
    # sectioned interval. (e.g., turning corner, section before/after stop sign, etc.)
    print("Before:", start_idx, end_idx)
    start_idx, end_idx = section_idx(start_idx, end_idx)
    print("After:", start_idx, end_idx)
    gaze_start_idx, gaze_end_idx = section_idx(gaze_start_idx, gaze_end_idx)
    print()

    # grab gaze data
    print("ts:", df_vehicle['time_real'][norm_idx])
    print("ts:", df_gaze['gaze_timestamp'][gaze_norm_idx])
    time = df_vehicle['time_real'][start_idx:end_idx] - df_vehicle['time_real'][norm_idx]
    gaze_time = df_gaze['gaze_timestamp'][gaze_start_idx:gaze_end_idx] - df_gaze['gaze_timestamp'][gaze_norm_idx]

    # update with sections
    # start_idx = time[(time > -10) & (time < 3)].index[0]
    # end_idx = time[(time > -10) & (time < 3)].index[-1]
    condition = (df_vehicle['ego_x'] < 100) & (df_vehicle['ego_y'] > 60)
    indices = df_vehicle.index[condition]
    start_index = indices.min()
    end_index = indices.max()
    time = df_vehicle['time_real'][start_idx:end_idx] - df_vehicle['time_real'][norm_idx]

    gaze_start_idx = gaze_time[(gaze_time > -10) & (gaze_time < 3)].index[0]
    gaze_end_idx = gaze_time[(gaze_time > -10) & (gaze_time < 3)].index[-1]
    gaze_time = df_gaze['gaze_timestamp'][gaze_start_idx:gaze_end_idx] - df_gaze['gaze_timestamp'][gaze_norm_idx]

    # Column data
    brake_pressure = df_vehicle['Brake[Bar]_avg'][start_idx:end_idx]
    v_x            = df_vehicle['v_x_Vehicle_CoG[km|h]'][start_idx:end_idx]
    a_y            = df_vehicle['a_y_Vehicle_CoG[m|s2]'][start_idx:end_idx]
    yaw            = df_vehicle['YawRate_Vehicle_CoG[deg|s]'][start_idx:end_idx]
    vline          = 0 # df_vehicle['time_real'][norm_idx]
    hline          = 0.5 

    # Plot variables
    YTICKS = [0.25, 0.375, 0.5, 0.625, 0.75] 

    # Plot brake pressure
    axs[i, 0].plot(time, brake_pressure, label=f'Lap {i+1}')
    axs[i, 0].set_title(f'Lap {i+1} - stop sign interval braking behavior')
    axs[i, 0].axvline(x=vline, color='red', linestyle='--')
    ax2 = axs[i, 0].twinx()
    norm_pos_x = df_gaze['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', s=0.2, alpha=0.5)
    ax2.axhline(y=hline, color='g', linestyle='--')
    ax2.set_ylabel('Gaze norm_pos_x')
    ax2.set_ylim(0.25, 0.75)
    ax2.set_yticks(YTICKS)
  
    # Plot velocity
    axs[i, 1].plot(time, v_x, label=f'Lap {i+1}')
    axs[i, 1].set_title(f'Lap {i+1} - X-direction velocity of Vehicle CoG')
    axs[i, 1].axvline(x=vline, color='red', linestyle='--')
    ax2 = axs[i, 1].twinx()
    norm_pos_x = df_gaze['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', s=0.2, alpha=0.5)
    ax2.axhline(y=hline, color='g', linestyle='--')
    ax2.set_ylabel('Gaze norm_pos_x')
    ax2.set_ylim(0.25, 0.75)
    ax2.set_yticks(YTICKS)

    # Plot acceleration
    axs[i, 2].plot(time, a_y, label=f'Lap {i+1}')
    axs[i, 2].set_title(f'Lap {i+1} - Y-direction acceleration of Vehicle CoG')
    axs[i, 2].axvline(x=vline, color='red', linestyle='--')
    ax2 = axs[i, 2].twinx()
    norm_pos_x = df_gaze['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', s=0.2, alpha=0.5)
    ax2.axhline(y=hline, color='g', linestyle='--')
    ax2.set_ylabel('Gaze norm_pos_x')
    ax2.set_ylim(0.25, 0.75)
    ax2.set_yticks(YTICKS)

    #Plot Yaw rate
    axs[i, 3].plot(time, yaw, label=f'Lap {i+1}')
    axs[i, 3].set_title(f'Lap {i+1} - Yaw rate of Vehicle CoG')
    axs[i, 3].axvline(x=vline, color='red', linestyle='--')
    ax2 = axs[i, 3].twinx()
    norm_pos_x = df_gaze['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', s=0.2, alpha=0.5)
    ax2.axhline(y=hline, color='g', linestyle='--')
    ax2.set_ylabel('Gaze norm_pos_x')
    ax2.set_ylim(0.25, 0.75)
    ax2.set_yticks(YTICKS)
    

    # Set common labels
    for j in range(len(column_labels)):
        axs[i, j].set_xlabel('Time (s)')

axs[5, 0].set_ylabel('Mean Brake Pressure Across 4 Wheels [Bar]', fontsize = 16)
axs[5, 1].set_ylabel('center of gravity x-velocity [Km/h]', fontsize = 16)
axs[5, 2].set_ylabel('center of gravity y-acceleration [m/(s^2)]', fontsize = 16)
axs[5, 3].set_ylabel('center of gravity yaw rate [deg/s]', fontsize = 16)

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

plt.savefig("time_veh_motion.png")